## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [37]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
    if agg['user__username__count'] >= 9:
         users.append(agg['user__username'])   
users

['jmperez', 'mkondra', 'ndebandi']

In [38]:
base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

50

¿Cuántos etiquetamos todes?

In [43]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Busquemos los que están etiquetados por todos

In [71]:
df = df[df.columns[df.notna().all()]]

df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?
name,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [73]:
coders.shape

(3, 10)

In [77]:
from nltk import agreement

coders = df.values

formatted_labels = []
for i in range(coders.shape[0]):
    # Acá las ponemos siguiendo el siguiente formato
    # [num etiquetador, num instancia, label]
    for j in range(coders.shape[1]):
        formatted_labels.append((i, j, coders[i, j]))
        

ratingtask = agreement.AnnotationTask(data=formatted_labels)

ratingtask.kappa()

0.4666666666666666

0.46, agreement moderado... Bueno, vamos viendo

In [83]:
ArticleLabel.objects.filter(
    article__title__in=df.columns[df.all()],
    user__username__in=users,
).count()

15